In [3]:
import os
import io
from pandas import DataFrame
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

def readFiles(path):
    for root, dirnames, filenames in os.walk(path):
        for filename in filenames:
            path = os.path.join(root, filename)

            inBody = False
            lines = []
            f = io.open(path, 'r', encoding='latin1')
            for line in f:
                if inBody:
                    lines.append(line)
                elif line == '\n':
                    inBody = True
            f.close()
            message = '\n'.join(lines)
            yield path, message


def dataFrameFromDirectory(path, classification):
    rows = []
    index = []
    for filename, message in readFiles(path):
        rows.append({'message': message, 'class': classification})
        index.append(filename)

    return DataFrame(rows, index=index)

data = DataFrame({'message': [], 'class': []})

path_spam = 'D:\Vincent\HETIC\Models_predict\emails\spam'
path_ham = 'D:\Vincent\HETIC\Models_predict\emails\ham'
df = pd.concat([dataFrameFromDirectory(path_spam, 'spam'),dataFrameFromDirectory(path_ham, 'ham')])


In [4]:
df.head()

,message,class
D:\Vincent\HETIC\Models_predict\emails\spam\00001.7848dde101aa985090474a91ec93fcf0,"<!DOCTYPE HTML PUBLIC ""-//W3C//DTD HTML 4.0 Tr...",spam
D:\Vincent\HETIC\Models_predict\emails\spam\00002.d94f1b97e48ed3b553b3508d116e6a09,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
D:\Vincent\HETIC\Models_predict\emails\spam\00003.2ee33bc6eacdb11f38d052c44819ba6c,1) Fight The Risk of Cancer!\n\nhttp://www.adc...,spam
D:\Vincent\HETIC\Models_predict\emails\spam\00004.eac8de8d759b7e74154f142194282724,##############################################...,spam
D:\Vincent\HETIC\Models_predict\emails\spam\00005.57696a39d7d84318ce497886896bf90d,I thought you might like these:\n\n1) Slim Dow...,spam


Our next step is to use CountVectorizer to split up each email message into a list of words and their counts.


In [5]:
from spacy.lang.en.stop_words import STOP_WORDS as en_stop #Spacy stopwords

Put the stopwords to remove common words

In [6]:
def remove_stopwords(text):
  stopwords = " ".join([word for word in str(text).split() if word not in en_stop]) #Spacy stopwords
  return stopwords

In [7]:
df["message"] = df["message"].apply(lambda x: remove_stopwords(x))

### Data Splitting

In [8]:
from sklearn.model_selection import train_test_split 

X = df["message"]
y = df["class"]

X_train , X_test, y_train, y_test = train_test_split(X,y,test_size=0.20)

### Training Dataset

In [9]:
vectorizer = CountVectorizer()
messages_train = X_train.values
counts_train = vectorizer.fit_transform(messages_train)

Training the model MultinomialNB

In [10]:
classifier = MultinomialNB()
classes_train = y_train.values
classifier.fit(counts_train, classes_train)

MultinomialNB()

In [11]:
test_msgs = ['$$$ Free Cash BANK', "Hey, what did you think of last night's episode of 'Friends'?"]
test_msg_counts = vectorizer.transform(test_msgs)

classifications = classifier.predict(test_msg_counts)
print(classifications) 

['spam' 'ham']


In [16]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(classifier, counts_train, classes_train, cv=5)

# Print the accuracy of each fold:
print(scores)

# Print the mean accuracy of all 5 folds
print(scores.mean())

[0.96875    0.97708333 0.95833333 0.98541667 0.94375   ]
0.9666666666666666


### Testing Dataset

In [13]:
messages_test = X_test.values
counts_test = vectorizer.fit_transform(messages_test)

classes_test = y_test.values
classifier.fit(counts_test, classes_test)

MultinomialNB()

In [14]:
test_msgs = ['$$$ Free Cash BANK', "Hey, what did you think of last night's episode of 'Friends'?"]
test_msg_counts = vectorizer.transform(test_msgs)

classifications = classifier.predict(test_msg_counts)
print(classifications) 

['spam' 'ham']


In [17]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(classifier, counts_test, classes_test, cv=5)

# Print the accuracy of each fold:
print(scores)

# Print the mean accuracy of all 5 folds
print(scores.mean())

[0.98333333 0.95       0.95       0.96666667 0.94166667]
0.9583333333333333
